<a href="https://colab.research.google.com/github/Mamtajangra/Question_answer_project-with-RNN-and-pytorch/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd

In [13]:
df = pd.read_csv("/content/100_Unique_QA_Dataset.csv")
df

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100
...,...,...
85,Who directed the movie 'Titanic'?,JamesCameron
86,Which superhero is also known as the Dark Knight?,Batman
87,What is the capital of Brazil?,Brasilia
88,Which fruit is known as the king of fruits?,Mango


In [14]:
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [15]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [16]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [17]:
# vocab because hmm jaanna chahte hai ki hmare dataset mai kitne unique words hai uske liye ek dictionary bnayenge
vocab = {'<UNK>':0}    ## UNK = unknown

In [18]:
# def build_vocab(row):
  # print(row["question"],row["answer"])

In [19]:
# df.apply(build_vocab, axis=1)

In [20]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer']) ## yha sab question and answer alag alag ho jaayenge means tokenize ho jaayenge
  # fir merge kardenge unko

  merged_tokens = tokenized_question + tokenized_answer   ##unko fir merge kiya ek list main laane ke liye
  # fir loop lgaya taakihar ek token ka length detect ho jo vocab main nhi h

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [21]:
def build_vocab(row):
  print(row["question"],row["answer"])

In [22]:
df.apply(build_vocab, axis=1)

What is the capital of France? Paris
What is the capital of Germany? Berlin
Who wrote 'To Kill a Mockingbird'? Harper-Lee
What is the largest planet in our solar system? Jupiter
What is the boiling point of water in Celsius? 100
Who painted the Mona Lisa? Leonardo-da-Vinci
What is the square root of 64? 8
What is the chemical symbol for gold? Au
Which year did World War II end? 1945
What is the longest river in the world? Nile
What is the capital of Japan? Tokyo
Who developed the theory of relativity? Albert-Einstein
What is the freezing point of water in Fahrenheit? 32
Which planet is known as the Red Planet? Mars
Who is the author of '1984'? George-Orwell
What is the currency of the United Kingdom? Pound
What is the capital of India? Delhi
Who discovered gravity? Newton
How many continents are there on Earth? 7
Which gas do plants use for photosynthesis? CO2
What is the smallest prime number? 2
Who invented the telephone? Alexander-Graham-Bell
What is the capital of Australia? Canber

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [23]:
df.apply(build_vocab, axis=1)

What is the capital of France? Paris
What is the capital of Germany? Berlin
Who wrote 'To Kill a Mockingbird'? Harper-Lee
What is the largest planet in our solar system? Jupiter
What is the boiling point of water in Celsius? 100
Who painted the Mona Lisa? Leonardo-da-Vinci
What is the square root of 64? 8
What is the chemical symbol for gold? Au
Which year did World War II end? 1945
What is the longest river in the world? Nile
What is the capital of Japan? Tokyo
Who developed the theory of relativity? Albert-Einstein
What is the freezing point of water in Fahrenheit? 32
Which planet is known as the Red Planet? Mars
Who is the author of '1984'? George-Orwell
What is the currency of the United Kingdom? Pound
What is the capital of India? Delhi
Who discovered gravity? Newton
How many continents are there on Earth? 7
Which gas do plants use for photosynthesis? CO2
What is the smallest prime number? 2
Who invented the telephone? Alexander-Graham-Bell
What is the capital of Australia? Canber

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [24]:
vocab  ## aise aaya ki what ko 1 label mila aise chlta jaa rha hai ye

{'<UNK>': 0}

In [25]:
len(vocab)

1

In [26]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:         ## agar wo token vocab main hai to append kar do nahi to unknown dedo isko
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [27]:
text_to_indices("What is Mamtu", vocab)   ## what = 1, is = 2 , mamtu = 0 kyunki mamtu isme hai hi nahi to unknown ye zero ho gaya

[0, 0, 0]

In [28]:
import torch   # hme har question ko same aise hi karna hai ki to hmm help lenge dataset aur dataloader ki
from torch.utils.data import Dataset, DataLoader

In [29]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):
## ye hmm dataset se pehle nikalenge ki konse konse row chahiye uss row ka answer and question
    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [30]:
dataset = QADataset(df, vocab)

In [31]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)   ## padding ki need nhi h kyunki ek batch main ek question answer jaa rha hai soo uskio need hoti h jb batch size 1 se jyada ho

In [32]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 0, 0, 0, 0, 0, 0, 0]]) tensor([0])
tensor([[0, 

In [33]:
import torch.nn as nn        ## hmm har word ka embedding krrhe hai aur uski dimension hmm 50 rakhkenge
# hidden main 60 neuron rakhenge ad output main 322

In [34]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()    ## yha hmm sequential nhi likhenge kyunki hme input ek ek nhi chahiye isme 1 se jyada input aati hai
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50) ## ye har layer ko 50 dimension vector main convert kar rha hai
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):   ## sequential nhi likha hmm yha pe manual forward pass likhenge
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))    ## fc = fully connected

    return output

In [35]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [36]:
learning_rate = 0.001
epochs = 20

In [37]:
model = SimpleRNN(len(vocab))

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [39]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 0.000000
Epoch: 2, Loss: 0.000000
Epoch: 3, Loss: 0.000000
Epoch: 4, Loss: 0.000000
Epoch: 5, Loss: 0.000000
Epoch: 6, Loss: 0.000000
Epoch: 7, Loss: 0.000000
Epoch: 8, Loss: 0.000000
Epoch: 9, Loss: 0.000000
Epoch: 10, Loss: 0.000000
Epoch: 11, Loss: 0.000000
Epoch: 12, Loss: 0.000000
Epoch: 13, Loss: 0.000000
Epoch: 14, Loss: 0.000000
Epoch: 15, Loss: 0.000000
Epoch: 16, Loss: 0.000000
Epoch: 17, Loss: 0.000000
Epoch: 18, Loss: 0.000000
Epoch: 19, Loss: 0.000000
Epoch: 20, Loss: 0.000000


In [40]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [41]:
predict(model, "What is the largest planet in our solar system?")

<UNK>
